In [1]:
import sys
sys.path.append("/home/sidhu/Projects/tf-transformers/src/")

In [63]:
import tempfile
from transformers import AlbertTokenizer

from tf_transformers.text.sentencepiece_layer import extend_sentencepicemodel
from tf_transformers.text import SentencepieceTokenizer

In [64]:
# We will use T5 tokenizer , but we extend it with 2 more tokens

# 1. [CLS]
# 2. [MASK]
# We will use </s> as SEP token

temp_dir = tempfile.mkdtemp()
model_name = 'albert-base-v2'
tokenizer_hf = AlbertTokenizer.from_pretrained(model_name)
save_path = "{}/{}".format(temp_dir, model_name)
tokenizer_hf.save_pretrained(save_path)

in_file  = '{}/spiece.model'.format(save_path)
# out_file = '{}/new_spiece.model'.format(save_path)
# special_tokens = ['[CLS]', '[MASK]']

# extend_sentencepicemodel(in_file, out_file, special_tokens)

init inputs ()
init kwargs {'model_max_length': 512, 'vocab_file': '/home/sidhu/.cache/huggingface/transformers/10be6ce6d3508f1fdce98a57a574283b47c055228c1235f8686f039287ff8174.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d', 'special_tokens_map_file': None, 'tokenizer_file': '/home/sidhu/.cache/huggingface/transformers/828a43aa4b9d07e2b7d3be7c6bc10a3ae6e16e8d9c3a0c557783639de9eaeb1b.670e237d152dd53ef77575d4f4a6cd34158db03128fe4f63437ce0d5992bac74', 'name_or_path': 'albert-base-v2'}


In [4]:
# To use new sentencepiece model in T5 use like this
t5_kwargs = {'bos_token': '[CLS]',
 'eos_token': '</s>', 
 'unk_token': '<unk>', 
 'pad_token': '<pad>', 
 'mask_token': '[MASK]', 
 'name_or_path': '{}'.format(save_path), 
 'vocab_file': '{}/new_spiece.model'.format(save_path)}
tokenizer_hf = T5Tokenizer(**t5_kwargs)
tokenizer_hf.unique_no_split_tokens = tokenizer_hf.all_special_tokens

In [78]:

# Use this one for fast processinf
tokenizer_tf = SentencepieceTokenizer(
        model_file_path=in_file,
        lower_case=True,
        special_tokens=['[CLS]', '[MASK]', '<unk>', '<pad>']
    )
print("total vocab is", tokenizer_tf.vocab_size)

total vocab is tf.Tensor(30000, shape=(), dtype=int32)


In [79]:
text = 'This is [MASK] token with [CLS] </s>'

In [80]:
tokenizer_hf.tokenize(text)

['▁this',
 '▁is',
 '[MASK]',
 '▁to',
 'ken',
 '▁with',
 '[CLS]',
 '▁',
 '<',
 '/',
 's',
 '>']

In [81]:
tokenizer_sp = tokenizer_tf._tokenizer

In [87]:
text = 'this is [MASK] token with [CLS] <unk>'
tokenizer_sp.string_to_id(text.split())

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([1565,  403,    4,    1, 1410,    2,    1], dtype=int32)>

In [88]:
tokenizer_sp.string_to_id(text)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [83]:
tokenizer_sp.split(text.lower())

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([   48,    25,   636, 23265,   500,    20,  2853,    29,   636,
        5316,    18,   500,    13,     1, 17081,     1], dtype=int32)>

In [84]:
tokenizer_sp.detokenize([   48,    25,   636, 23265,   500,    20,  2853,    29,   636,
        5316,    18,   500,    13,     1, 17081,     1])

<tf.Tensor: shape=(), dtype=string, numpy=b'this is [mask] token with [cls]  \xe2\x81\x87 unk \xe2\x81\x87 '>

In [38]:
tokenizer_sp.detokenize([ 3713,   159, 32001,     2,  4065, 32000]).numpy().decode()

'Thisis[MASK] ⁇ with[CLS]'

In [43]:
tokenizer_sp.string_to_id(['▁This', '▁is', '[MASK]', '▁token', '▁with', '[CLS]', '</s>'])

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([  100,    19, 32001, 14145,    28, 32000,     1], dtype=int32)>

In [12]:
tokenizer_hf.encode(text)

[100, 19, 32001, 14145, 28, 32000, 1]

In [14]:
tokenizer_hf.decode([100, 19, 32001, 14145, 28, 32000, 1])

'This is[MASK] token with[CLS] </s>'

In [15]:
tokenizer_sp.detokenize([100, 19, 32001, 14145, 28, 32000, 1])

<tf.Tensor: shape=(), dtype=string, numpy=b'This is[MASK] token with[CLS]'>

In [16]:
tokenizer_hf.decode([ 3713,   159, 32001,     2,  4065, 32000])

'Thisis[MASK] <unk> with[CLS]'

In [17]:
tokenizer_sp.tokenize("This is")

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([100,  19], dtype=int32)>

In [48]:
from tf_transformers.text import sentencepiece_model_pb2


mp = sentencepiece_model_pb2.ModelProto()
mp.ParseFromString(open(in_file, "rb").read())

for token in special_tokens:
    new_token = sentencepiece_model_pb2.ModelProto().SentencePiece()
    new_token.piece = token
    new_token.score = 0
    new_token.type = 3
    mp.pieces.append(new_token)
    print(f'added {token}...')

with open(out_file, 'wb') as f:
    f.write(mp.SerializeToString())

added [CLS]...
added [MASK]...


In [49]:
import sentencepiece as sp

In [76]:
sp_model = sp.SentencePieceProcessor()
import tensorflow as tf
model_proto = tf.io.gfile.GFile(in_file, "rb").read()
sp_model.LoadFromSerializedProto(model_proto)

True

In [89]:
sp_model.encode_as_pieces(text)

['▁this',
 '▁is',
 '▁[',
 'MASK',
 ']',
 '▁to',
 'ken',
 '▁with',
 '▁[',
 'CLS',
 ']',
 '▁',
 '<',
 'unk',
 '>']

In [55]:
text = 'This is [MASK] token with [CLS] </s>'


In [56]:
tokenizer_sp.split(text)

<tf.Tensor: shape=(13,), dtype=int32, numpy=
array([  100,    19,     3, 32001, 14145,    28,     3, 32000,     3,
           2,    87,     7,  3155], dtype=int32)>

In [61]:
tokenizer_sp.detokenize([  100,    19,     3, 32001, 14145,    28,     3, 32000,     3,
           2,    87,     7,  3155])

<tf.Tensor: shape=(), dtype=string, numpy=b'This is [MASK] token with [CLS]  \xe2\x81\x87 /s>'>

In [ ]:
model_proto = tf.io.gfile.GFile(out_file, "rb").read()


In [90]:
import nltk

In [91]:
article = '''Wikipedia (/ˌwɪkɪˈpiːdiə/ (About this soundlisten) wik-ih-PEE-dee-ə or /ˌwɪki-/ (About this soundlisten) wik-ee-) is a free content, multilingual online encyclopedia written and maintained by a community of volunteer contributors through a model of open collaboration, using a wiki-based editing system. Wikipedia is the largest and most-read reference work in history,[3] and is consistently one of the 15 most popular websites as ranked by Alexa; as of 2021, it was ranked as the 13th most popular site.[3][4] The project carries no advertisements and is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through small donations.[5]

Wikipedia was launched on January 15, 2001, by Jimmy Wales[6] and Larry Sanger; Sanger coined its name as a blending of "wiki" and "encyclopedia".[7][8] Initially available only in English, versions in other languages were quickly developed. Combined, Wikipedia's editions comprise more than 56 million articles, that attract an average of around 2 billion unique device visits and receive more than 17 million edits per month, or develop at a rate of 1.9 edits per second.[9][10]

Wikipedia has received praise for its enablement of the democratization of knowledge, extent of coverage, unique structure, culture, and reduced amount of commercial bias, but criticism for exhibiting systemic bias, particularly gender bias against women.[11] Its reliability was frequently criticized in the 2000s, but has improved over time and has been generally praised in the late 2010s and 2020s.[12][3][11] Its coverage of controversial topics such as American politics and major events such as the COVID-19 pandemic has received substantial media attention. At various points, Wikipedia has been censored by world governments, ranging from the blocking of specific pages to bans on the entire site. Wikipedia has become an element of popular culture, with references in books, films and academic studies. In 2018, Facebook and YouTube announced that they would help users detect fake news by suggesting fact-checking links to related Wikipedia articles.[13][14]'''

In [95]:
for i in nltk.tokenize.sent_tokenize(article):
    print(i)
    print("-----------------------------------")

Wikipedia (/ˌwɪkɪˈpiːdiə/ (About this soundlisten) wik-ih-PEE-dee-ə or /ˌwɪki-/ (About this soundlisten) wik-ee-) is a free content, multilingual online encyclopedia written and maintained by a community of volunteer contributors through a model of open collaboration, using a wiki-based editing system.
-----------------------------------
Wikipedia is the largest and most-read reference work in history,[3] and is consistently one of the 15 most popular websites as ranked by Alexa; as of 2021, it was ranked as the 13th most popular site.
-----------------------------------
[3][4] The project carries no advertisements and is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through small donations.
-----------------------------------
[5]

Wikipedia was launched on January 15, 2001, by Jimmy Wales[6] and Larry Sanger; Sanger coined its name as a blending of "wiki" and "encyclopedia".
-----------------------------------
[7][8] Initially available only in 

In [93]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sidhu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True